In [2]:
import requests
import os
import csv
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from random import randint
from datetime import datetime
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# CSV öffnen: 
# Hausbesitzer mit Koordinaten

haus = pd.read_csv('Baurechtsnehmer_mit_Koordinaten.csv', sep=';')
                     


In [4]:
# Sortieren nach Grundstück-Nummer
haus.sort_values("NameVorname").head(10)

# Zeilen mit NaN in Grundstueck-Nummer löschen
haus = haus.dropna(axis=0, subset=['Name'])

# Sortieren nach Grundstück-Nummer
haus.sort_values("NameVorname").head(10)


,Grundstueck-Nummer,EGID,Gebaeude-Koordinate X,Gebaeude-Koordinate Y,Gebaeude-Strasse,Gebaeude-Hausnummer,Gebaeude-PLZ,Gebaeude-Ort,Koordinate,Grundstueck-Typ,Grundstueck-Flaeche,Eigentumsart,Name,Vorname,NameVorname,Adresse,PLZ,Ort,Datum der Abfrage
752,5838,009 001 194,2758012,1190339,Richtstrasse,NaN,7000,Chur,POINT (2758012 1190339),"SelbstRecht, Baurecht",2021.0,Alleineigentum,A. Käppeli's Söhne AG,NaN,A. Käppeli's Söhne AG,St. Gallenstrasse 72,7320,Sargans,2020-01-10 16:02:56.139407
1142,10109,160 016 236,2757919,1191496,Industriestrasse,NaN,7000,Chur,POINT (2757919 1191496),"SelbstRecht, Baurecht",4847.0,Alleineigentum,A. Vögele & Söhne Recycling AG,NaN,A. Vögele & Söhne Recycling AG,Industriestrasse 5A,7000,Chur,2020-01-11 01:08:45.573573
451,10109,009 001 093,2757860,1191471,Industriestrasse,NaN,7000,Chur,POINT (2757860 1191471),"SelbstRecht, Baurecht",4847.0,Alleineigentum,A. Vögele & Söhne Recycling AG,NaN,A. Vögele & Söhne Recycling AG,Industriestrasse 5A,7000,Chur,2020-01-11 01:08:45.573573
285,11679,160 013 662,2759811,1192611,Masanserstrasse,NaN,7000,Chur,POINT (2759811 1192611),"SelbstRecht, Baurecht",595.0,Alleineigentum,ARAI Immobilien AG,NaN,ARAI Immobilien AG,Felsenaustrasse 47,7000,Chur,2020-01-04 10:49:17.584321
454,5829,009 001 050,2757343,1190306,Emserstrasse,NaN,7000,Chur,POINT (2757343 1190306),"SelbstRecht, Baurecht",2195.0,Alleineigentum,ARGO Stiftung für Integration von Menschen mit...,NaN,ARGO Stiftung für Integration von Menschen mit...,Bahnhofstrasse 3,7001,Chur,2020-01-10 16:02:14.242052
864,5866,001 739 500,2758753,1190439,Kornquaderweg,NaN,7000,Chur,POINT (2758753 1190439),"SelbstRecht, Baurecht",2124.0,Stockwerkeigentum,ART fina AG,NaN,ART fina AG,Kornplatz 2,7000,Chur,2020-01-07 15:44:25.923376
1564,8954,009 001 238,2757719,1190768,Sommeraustrasse,NaN,7000,Chur,POINT (2757719 1190768),"SelbstRecht, Baurecht",5497.0,Stockwerkeigentum,AZ-AGVS Genossenschaft Ausbildungszentrum des ...,NaN,AZ-AGVS Genossenschaft Ausbildungszentrum des ...,Hinterm Bach 40,7000,Chur,2020-01-04 09:57:08.011758
1579,5985,001 739 490,2758582,1190333,Foralweg,NaN,7000,Chur,POINT (2758582 1190333),"SelbstRecht, Baurecht",2810.0,Stockwerkeigentum,Abeyesekere,Filomena,Abeyesekere Filomena,Foralweg 9,7000,Chur,2019-12-29 20:54:54.689471
908,10095,001 741 445,2758238,1191798,Föhrenweg,NaN,7000,Chur,POINT (2758238 1191798),"SelbstRecht, Baurecht",414.0,Miteigentum,Accola,Anna Luisa,Accola Anna Luisa,Föhrenweg 15,7000,Chur,2019-12-30 11:06:24.403442
1624,10047,001 741 463,2758143,1191846,Primelweg,NaN,7000,Chur,POINT (2758143 1191846),"SelbstRecht, Baurecht",341.0,Miteigentum,Accola,Daniel Markus,Accola Daniel Markus,Primelweg 3,7000,Chur,2019-12-30 10:43:28.355400


In [5]:
haus['Grundstueck-Flaeche'].median()

2124.0

In [6]:
haus['Grundstueck-Flaeche'].max()

80962.0

In [7]:
haus['Grundstueck-Flaeche'].min()

77.0

In [11]:
# die 50 grössten Baurecht-Besitzer von Chur (Anzahl Häuser)



nur_stwe = haus['Eigentumsart'] == 'Stockwerkeigentum'

nur_stwe = haus[nur_stwe]

nur_stwe.groupby('NameVorname')['Grundstueck-Nummer'].value_counts().sort_values(ascending=False).head(50)


NameVorname                 Grundstueck-Nummer
Shaqiri Bujar               9886                  4
Wilhelm Arno                9886                  4
Vlachos Petros              9886                  4
Vlachos Sofia               9886                  4
Hajredini Ursina            9886                  4
Deflorin Roman              9886                  4
Frangi Marion               9886                  4
Tarnutzer Andreas           9886                  4
Käslin Maria 'Prisca'       9886                  4
Frischknecht Helene         9886                  4
Fritsch Johanna             9886                  4
Wilhelm Martina             9886                  4
Kuratle Claudia Karin       9886                  4
Capol Monika Anna Maria     9886                  4
Kuratle Brigitte            9886                  4
Hitz Leo                    9886                  4
Christen Verena             9886                  4
Müller Ralph Michael        9886                  4
Shaqiri Eljesa   

In [ ]:
# diese Auswertung ist nutzlos, da Stockwerkeigentum nicht nur die eigentliche Wohnung umfasst, sondern auch Gemeinschaftsräume etc. 
# Eine hohe Anzahl hier sagt also nichts über den tatsächlichen Besitz von Baurechten aus.